In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoTokenizer
import pandas as pd

In [9]:
# 커스텀 모델 클래스 정의
class MultiLabelClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = AutoModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(0.1)
        self.classifier_도수 = nn.Linear(768, 3)  # 낮은, 중간, 높은
        self.classifier_술종류 = nn.Linear(768, 4)  # 칵테일, 럼, 위스키, 보드카
        self.classifier_맛 = nn.Linear(768, 5)  # 달달한, 쓴, 상큼한, 신맛, 부드러운

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[0][:, 0, :]  # CLS 토큰의 출력
        pooled_output = self.dropout(pooled_output)
        
        return {
            '도수': self.classifier_도수(pooled_output),
            '술종류': self.classifier_술종류(pooled_output),
            '맛': self.classifier_맛(pooled_output)
        }

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 데이터 로드
data = pd.read_csv(r"C:\Users\user\OneDrive\바탕 화면\project\OSS_Project\AI\text-model\data\processed_data.csv")
inputs = tokenizer(list(data['입력 문장']), padding=True, truncation=True, return_tensors="pt")

# 레이블 매핑
도수_매핑 = {'낮은': 0, '중간': 1, '높은': 2}
술종류_매핑 = {'칵테일': 0, '럼': 1, '위스키': 2, '보드카': 3}
맛_매핑 = {'달달한': 0, '쓴맛': 1, '상큼한': 2, '신맛': 3, '부드러운': 4}

# 레이블 변환
도수_labels = torch.tensor([도수_매핑[도수] for 도수 in data['도수']])
술종류_labels = torch.tensor([술종류_매핑[종류] for 종류 in data['술 종류']])
맛_labels = torch.tensor([맛_매핑[맛] for 맛 in data['맛']])

# 데이터셋 생성
dataset = torch.utils.data.TensorDataset(
    inputs['input_ids'], 
    inputs['attention_mask'], 
    도수_labels,
    술종류_labels,
    맛_labels
)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# 모델 초기화
model = MultiLabelClassifier()
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# 학습 루프
for epoch in range(3):
    model.train()
    for batch in dataloader:
        input_ids, attention_mask, 도수_label, 술종류_label, 맛_label = batch
        
        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        # 각 분류기의 손실 계산
        도수_loss = loss_fn(outputs['도수'], 도수_label)
        술종류_loss = loss_fn(outputs['술종류'], 술종류_label)
        맛_loss = loss_fn(outputs['맛'], 맛_label)
        
        # 전체 손실 계산
        total_loss = 도수_loss + 술종류_loss + 맛_loss
        
        # Backward pass
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        
        print(f"Epoch {epoch}, Loss: {total_loss.item()}")

# 모델 저장
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, "models/bert_model/model.pt")
tokenizer.save_pretrained("bert_model/")
print("모델 저장 완료")

Epoch 0, Loss: 4.68547248840332
Epoch 0, Loss: 3.8798367977142334
Epoch 0, Loss: 4.586738586425781
Epoch 0, Loss: 4.591320514678955
Epoch 0, Loss: 4.757419586181641
Epoch 0, Loss: 4.0418572425842285
Epoch 0, Loss: 4.290745258331299
Epoch 0, Loss: 4.452389240264893
Epoch 0, Loss: 4.3101806640625
Epoch 0, Loss: 3.9708962440490723
Epoch 0, Loss: 3.8853495121002197
Epoch 0, Loss: 4.67462158203125
Epoch 0, Loss: 3.611929416656494
Epoch 1, Loss: 4.214059829711914
Epoch 1, Loss: 3.9474401473999023
Epoch 1, Loss: 4.104005813598633
Epoch 1, Loss: 3.7866482734680176
Epoch 1, Loss: 3.8137824535369873
Epoch 1, Loss: 5.470815658569336
Epoch 1, Loss: 4.548715114593506
Epoch 1, Loss: 4.430315017700195
Epoch 1, Loss: 4.050725936889648
Epoch 1, Loss: 4.201711177825928
Epoch 1, Loss: 4.132125377655029
Epoch 1, Loss: 4.154841899871826
Epoch 1, Loss: 3.373337984085083
Epoch 2, Loss: 3.4883337020874023
Epoch 2, Loss: 3.9418561458587646
Epoch 2, Loss: 4.261086463928223
Epoch 2, Loss: 5.368455410003662
Epoch

RuntimeError: Parent directory models/bert_model does not exist.